### Intro

> Diederik P. Kingma와 Max Welling의 2013년 논문인 'Auto-Encoding Variational Bayes'를 리뷰한다.
- 이 논문에서는 예를들어 원인이 되는 연속형 변수 $z$와 결과가 되는 $x$가 존재하는 상황(directed probabilistic model)에서 $p(z \mid x)$를 구하기 어려운 문제를 풀기 위한 확률적 변분 추론&학습(stochastic variational inference & learning) 방법을 제시한다. 이를 위해 새로운 형태의 variational lower bound와 이를 이용한 학습 방법을 제안한다.

 
> - 연속형 잠재 변수를 포함하는 방향성 확률 모형에 대한 일반적인 접근방법은 변분 베이지안 방법을 이용하는 것인데, 이는 구하기 어려운 사후 분포에 대한 근사분포를 구하는 방식이다. 하지만 일반적으로 사용되는 mean-field를 이용한 변분 베이지안 방법은 근사 사후분포에 대한 기댓값을 해석적으로 구할 수 있어야 한다는 제약사항이 있다. 이 논문에서는 mean-field 방법에서와 같은 분포에 대한 가정 없이 lower bound에 대한 미분 가능한 불편 추정량(SGVB:Stochastic Gradient Variational Bayes)을 제시한다.

> - 또한 '각 데이터 샘플마다 잠재 변수가 존재하는 i.i.d 데이터셋'에 적용 가능한 AEVB:Auto-Encoding VB 알고리즘을 제시할텐데 이건 SGVB를 이용해서 신경망과 같은 recognition model을 이용해 매우 효율적으로 추론과 학습을 할 수 있게 해 준다. 기존에 이런 류의 방법에서는 각 데이터 샘플마다 MCMC와 같은 방법으로 많은 수의 샘플링을 해야했지만 이 방법에서는 단지 ancestral sampling으로 그리 많지 않은 샘플링 만으로도 학습이 가능하다. recognition model로 신경망을 사용하면 그 모형을 VAE(Variational auto-encoder)라 할 것이다.

### Information, Entropy and KL divergence

> 그래서 뭘 어떻게 하겠다는 건가?
- 기본적으로 우리가 하려는 것은 잠재변수가 연속형인 directed probabilistic model에서 추론과 학습 문제를 다룬다. 이런 문제에서 결국 해야하는 것은 어떤 어려운 분포 $p(z \mid x)$를 구하는 것인데, 여기서는 새로운 형태의 Variational Bayesian 방법을 사용할 것이다. 

> - 우선 Variational Bayesian method에 대해 간단히 알아본다.

> - 당연히 일어날 일에 대한 소식은 별 정보가 아니다. 반면 일어날 것 같지 않은 일에 대한 소식은 큰 정보라 할 수 있다. 즉 사건이 발생할 확률에 반비례하계 **정보량**을 정의할 수 있다.
$$I = - \log p(x)$$


> - 평균 정보량을 엔트로피(entropy)라 한다.
$$H = \mathbb{E}[I] =  - \sum p(x) \log p(x)$$

> - 분포 $p$를 기준으로한 **분포 $q$의 KL divergence**(KL div $q$ w.r.t $p$), $KL(p \Vert q)$를 정보량&엔트로피 관점에서 생각해볼 수 있다. 
 - 우선 분포 $p$와 분포 $q$의 엔트로피 차이는 아래와 같다.
 $$\begin{align}
 -\sum \color{red}{q(x)} \log q(x) + \sum p(x) \log p(x)
 \end{align}$$
 - 위 식에서 기댓값의 기준 확률을 $p$로 통일하면 아래와 같이 $KL(p \Vert q)$가 된다.
 $$\begin{align}
 & -\sum \color{red}{p(x)} \log q(x) + \sum p(x) \log p(x) \\
 & = \sum_x p(x) \log \frac{p(x)}{q(x)} \\
 & = KL(p \Vert q)
 \end{align}$$

### Variational Inference - ELBO
> 참고
- https://ermongroup.github.io/cs228-notes/inference/variational/
- David Barber, Bayesian Reasoning and Machine Learning p619

> - 어떤 미지의 확률분포 $p$에 근접한 분포 $q$를 추론하는 문제가 있다고 할 때, 이를 풀 수 있는 방법들중 하나가 Variational inference이다. 결론부터 말하자면 분포에 대한 추론(inference) 문제를 최적화(optimization) 문제로 바꿔 해결하는 방법이다. 이런 문제를 MCMC와 같은 샘플링으로도 접근할 수 있으나 샘플링을 이용하는 경우 얻은 솔루션이 얼마나 $p$에 가까운지 알기 어렵고, 적절한 샘플링 기법을 선택하는 것 자체가 어렵기도 하다.
- Variational inference에서는 두 분포의 KL divergence를 이용한다. 참고로 이산 분포 $p$와 $q$의 KL divergence는 아래와 같이 정의된다.
 - 이 값은 $p$와 $q$에 무관하게 항상 0보다 크거나 같고, 만약 두 분포가 같다면 이 값은 0이 된다.
$$\begin{align}
KL(q \Vert p) &= \sum_x q(x) \log \frac{q(x)}{p(x)} \\
&= \mathbb{E}_{q(x)} \bigg[ \log q(x) - \log p(x) \bigg]
\end{align}$$


> - 관측 데이터 $x_1, \cdots x_n$에 대한 likelihood를 구하기 위해 베이지안 접근법에서는 아래와 같은 방식을 취한다.
$$p(\mathbf{x}) = \int_\mathbf{z} p(\mathbf{x} \mid \mathbf{z}) p(\mathbf{z}) ~ d\mathbf{z}$$
 하지만 latent $\mathbf{z}$가 고차원인경우 이 적분을 계산하는 것이 어려워진다. 대신 아래와 같이 베이즈룰과 Kullback-Leibler divergence를 이용한 근사를 이용할 수 있다.

> - 만약 $p(\mathbf{x} \mid \mathbf{z})$를 $q(\mathbf{z})$로 근사한다면, 베이즈룰을 이용해 두 분포의 차이를 아래와 같이 표현할 수 있다.
$$\begin{align}
KL\big( q(\mathbf{z}) \Vert p(\mathbf{z} \mid \mathbf{x}) \big) &= \mathbb{E}_{q(\mathbf{z})} \bigg[ \log q(\mathbf{z}) - \log p(\mathbf{z} \mid \mathbf{x}) \bigg] \\
&= \mathbb{E}_{q(\mathbf{z})} \bigg[ \log q(\mathbf{z}) - \log p(\mathbf{x} \mid \mathbf{z}) p(\mathbf{z}) \bigg] + \log p(\mathbf{x}) \\
&\because p(\mathbf{z} \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid \mathbf{z}) p(\mathbf{z})}{p(\mathbf{x})}
\end{align}$$

> - KL divergence는 항상 0보다 크거나 같으므로 아래와 같이 $\log p(\mathbf{x})$의 lower bound를 구할 수 있다. 이를 variational lower bound 혹은 evidence lower bound(**ELBO**)라 한다.
$$\begin{align}
\log p(\mathbf{x}) &= KL\big( q(\mathbf{z}) \Vert p(\mathbf{z} \mid \mathbf{x}) \big) - \mathbb{E}_{q(\mathbf{z})} \bigg[ \log q(\mathbf{z}) - \log p(\mathbf{x} \mid \mathbf{z}) p(\mathbf{z}) \bigg] \\
&\geq \mathbb{E}_{q(\mathbf{z})} \bigg[ \log p(\mathbf{x} \mid \mathbf{z}) p(\mathbf{z}) - \log q(\mathbf{z}) \bigg] \\
&= \mathbb{E}_{q(\mathbf{z})} \bigg[ \log p(\mathbf{x} \mid \mathbf{z}) - \log \frac{q(\mathbf{z})}{p(\mathbf{z})} \bigg] \\
&= \mathbb{E}_{q(\mathbf{z})} \bigg[ \log p(\mathbf{x} \mid \mathbf{z}) \bigg] - KL\big( q(\mathbf{z}) \Vert p(\mathbf{z}) \big)
\end{align}$$

> - 이런 lower bound는 어떤 형태의 $q(\mathbf{z})$에 대해서도 성립하며 근사함수 $q(\mathbf{z})$가 근사하려는 함수 $p(\mathbf{z} \mid \mathbf{x})$과 같다면 양변은 같아진다. lower bound의 두 항($\mathbb{E}[\cdot]$, $KL(\cdot)$)을 계산하는 하는 것이 용이하도록 $q(\mathbf{z})$를 선택하는 것이 중요하며, 함수 $q(\mathbf{z})$의 여러 파라미터를 잘 조절하여 최대한 lower bound가 커지도록 해야한다.

> - 그렇다면 어떤 형태의 $q$함수를 사용해야할까? 지수족 함수, 신경망, 가우시안 과정, 잠재변수 모형 등 다양한 선택지가 있다. 이들 중 아래와 같은 형태의 $q(\mathbf{z})$를 사용하는 방법을 'Mean-field inference'라 한다. (여기서 $q_i(x_i)$는 이산형 확률변수 $x_i$에 대한 확률분포이다.)
$$q(\mathbf{z}) = q_1(z_1) q_2(z_2) \cdots q_n(z_n)$$
- Mean-field inferece는 아래 목적함수를 최소화 하는 식으로 풀게 된다. (이를 푸는 방법으로 $q_j$를 하나식 선택하고 나머지 $q_{\cdot}$는 고정한체로 $J(q)$를 최소화하는 $q_j$를 찾을 수 있는데, 이런 방법을 coordinate descent라 한다.)
$$\min_{q_1, \cdots, q_j, \cdots, q_n}J(q)$$

In [ ]:
# deplicated

'''
### Variational Auto-Encoder
> 참고
- https://ermongroup.github.io/cs228-notes/extras/vae/ 'Auto-encoding variational Bayes' ~

> 데이터셋 $D = \{ x^1, x^2, \cdots, x^n \}$이 있을 때 아래와 같은 문제를 푼다고 해 보자.
1. 분포 $p$의 파라미터 $\theta$를 학습시키고 싶다.
2. $x$가 주어졌을 때 $z$에 대한 사후분포를 알고 싶다.(inference)
3. 부분적으로 유실된 이미지 $x$가 주어졌을 때 유실된 부분을 체우고 싶다.

> 이런 문제에는 아래와 같은 상황을 가정한다.
> - $p(z \mid x)$를 구하는 것이 쉽지 않다.
- 데이터셋 $D$가 굉장히 커서 한번에 처리하기 어렵다. subsample-batch를 사용해야한다.

> - (결론만 말하면) EM algorithm, mean fiend inference, sampling-based methods 등으로는 위 문제를 풀기 어렵다.

> - 이 논문에서는 위 문제를 푸는데 Auto-encoding variational bayes(AEVB)를 이용한다. 앞서 다뤘다시피 AEVB는 variational inference에 기반하는데 이는 Evidence lower bound(ELBO)를 최대화하는 접근법이다. ELBO는 아래와 같다. (앞서 $\mathbb{E}_{q(\theta)} \big[ \log p(x_1, \cdots, x_n \mid \theta) p(\theta) - \log q(\theta) \big]$와 같은 식이다.)
$$\mathcal{L}(\theta, \phi; x) = \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z \mid x)\bigg]$$

> - Furthermore, instead of only doing inference, we will simultaneously perform learning via gradient descent on both ϕ and θ (jointly). Optimization over ϕ will keep ELBO tight around logp(x); optimization over θ will keep pushing the lower bound (and hence logp(x)) up. This is somewhat similar to how the EM algorithm optimizes a lower bound on the marginal likelihood.
'''

"""
> $$\begin{align}
KL \big(q(z \mid x) \Vert p(z \mid x) \big) &= \sum_z q(z \mid x) \log \frac{q(z \mid x)}{p(z \mid x)} \\
&= \sum_z q(z \mid x) \log \big( \frac{q(z \mid x)}{p(x, z)} p(x) \big)\\
&= \sum_z q(z \mid x) \big( \log \frac{q(z \mid x)}{p(x, z)} + \log p(x) \big)\\
&= KL\big( q(z \mid x) \Vert p(x, z) \big) + \log p(x) \\
\end{align}$$

> $$\begin{align}
\log p(x) &= KL \big(q(z \mid x) \Vert p(z \mid x) \big) - KL\big( q(z \mid x) \Vert p(x, z) \big)\\
&= KL \big(q(z \mid x) \Vert p(z \mid x) \big) + \mathbb{E}_{q_\theta (z \mid x)} \bigg[ \log p(x, z) - \log q(z \mid x)\bigg]
\end{align}$$

> - $x$는 evidence로 주어져 있으므로 $\log p(x)$는 상수이므로 $KL \big(q(z \mid x) \Vert p(z \mid x) \big)$를 최소화 하는 것은 $\mathbb{E}_{q_\theta (z \mid x)} \bigg[ \log p(x, z) - \log q(z \mid x)\bigg]$를 최대화 하는 것과 같다. 결국 evidence lower bound(ELBO)를 목적함수로 두고 이를 최대화 할 것이다. ($p$는 $\theta$로, $q$는 $\phi$로 parameterized되어 있다면 ELBO를 아래와 같이 표현할 수 있다.)
$$\mathcal{L}(p_\theta, q_\phi) = \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z \mid x)\bigg]$$
- 혹은 위 논의에서 $x$는 주어져 있으므로 $p(z \mid x)$에 대한 근사 함수로 $q(z \mid x)$ 대신 $q(z)$를 사용할수도 있다. 그 경우 ELBO는 아래와 같다.
$$\mathcal{L}(p_\theta, q_\phi) = \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z)\bigg]$$
"""

> - ELBO를 아래와 같이 다시 표현할 수 있다.
$$\begin{align}
\mathcal{L}(\theta, \phi; x) &= - KL\big( q(z \mid x) \Vert p(x, z) \big) = \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z \mid x)\bigg] \\
&= \sum_z q_\phi (z \mid x) \log \frac{p_\theta (x, z)}{q_\phi (z \mid x)}\\
&= \sum_z q_\phi (z \mid x) \log \frac{p_\theta (x \mid z) p_\theta(z)}{q_\phi (z \mid x)}\\
&= \sum_z q_\phi (z \mid x) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z \mid x)} \bigg)\\
&= \sum_z q_\phi (z \mid x) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z \mid x)} \bigg)\\
&= \mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg] - KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)
\end{align}$$

> - 혹은 아래와 같다.
$$\begin{align}
\mathcal{L}(\theta, \phi; x) &= - KL\big( q(z) \Vert p(x, z) \big) = \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z)\bigg] \\
&= \sum_z q_\phi (z) \log \frac{p_\theta (x, z)}{q_\phi (z)}\\
&= \sum_z q_\phi (z) \log \frac{p_\theta (x \mid z) p_\theta(z)}{q_\phi (z)}\\
&= \sum_z q_\phi (z) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z)} \bigg)\\
&= \sum_z q_\phi (z) \bigg( \log p_\theta (x \mid z) + \log \frac{p_\theta(z)}{q_\phi (z)} \bigg)\\
&= \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x \mid z) \bigg] - KL\big (q_\phi (z) \Vert p_\theta(z) \big)
\end{align}$$

> - **'ELBO를 최대화한다는 것'**의 의미를 다시 생각해보면             
 - $\mathbb{E}_{q_\phi (z \mid x)} \bigg[ \log p_\theta (x \mid z) \bigg]$는 $z$라는 latent가 주어졌을 때 데이터 $x$를 관찰할 log-likelihood이고 이를 최대화 하는 것이다.                      
  - 반면 $KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)$는 $q(z)$와 $p(z)$의 차이이며 이를 최소화 해야한다.

> - VAE에서는... (사실 아직 VAE를 설명하지 않았다.)
 - $q_\phi(z \mid x)$는 encoder network, $p_\theta(x \mid z)$는 decoder network이고 $p(z)$는 $\mathcal{N}(0, 1)$이다.
 - $\mathbb{E}_{q_\phi (z \mid x)} \big[ \log p_\theta (x \mid z) \big]$를 expected negative **reconstruction error**이고, 이를 최대화 하는 것은 입력 $x$와 decoder의 출력 $\hat{x}$와의 차이를 최소화 하는 것이 된다.
 - $KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)$는 regularization term 역할을 하며, 이를 최소화 하는 것은 encoder의 출력 $q(z \mid x)$를 최대한 $p(z) = \mathcal{N}(0, 1)$에 가깝게 만드는 것이다.

### The reparametrization trick and SGVB

> - 샘플링이 어려운 어떤 함수 $q_\phi (z \mid x)$에서 샘플링하는 트릭중 하나가 reparameterization이다. 예를들어 $z \sim \mathcal{N}(3+x, 2)$에서 샘플링하는 것이 (그렇지는 않지만..)어렵다면,  $\mathcal{N}(0, 1)$에서 $\epsilon$을 뽑고, $2\epsilon + 3 + x$를 계산해서 $z$에 대한 샘플을 대체할 수 있다.
- 이처럼 $q_\phi(z \mid x)$에서 $z$를 샘플링하기 어려울 때, $z$대신 $\epsilon$을 샘플링하고 $z, x, \epsilon$의 관계를 표현하는 어떤 미분가능한 함수 $z=g_\phi(\epsilon, x)$를 이용해 $z$에 대한 샘플을 $g_\phi(\epsilon, x)$로 대체한다.(위 예에서는 $z=g(\epsilon, x)=2\epsilon + 3 + x$) 이때 $g_\phi$는 $\phi$로 parameterized된 어떤 미분가능한 함수이고, $p(\epsilon)$는 $z$와 독립적인 분포이다.

> - reparameterization trick을 사용하면 $q_\phi(z \mid x)$에 대한 기댓값을 샘플링으로 계산 가능한 형태로 변형할 수 있다. (이전 이후 수식에서 $x, z, \epsilon$ 모두 vector notation으로 바껴야 함)
$$\begin{align}
\mathbb{E}_{q_\phi(z \mid x)} \big[ f(z) \big] &= \int q_\phi(z \mid x) f(z) dz \\
&=\int p(\epsilon) f(z) d\epsilon \\
&=\int p(\epsilon) f(g_\phi(\epsilon, x)) d\epsilon \\
&\simeq \frac{1}{L} \sum_{l=1}^L f(g_\phi(x, \epsilon^{(l)})) ~~~ \text{where} ~~ \epsilon^{l} \sim p(\epsilon)
\end{align}$$

> - 앞서 ELBO는 아래와 같았다. 여기에도 위 트릭을 적용해서 sample average로 계산된다. 이런 lower bound를 **SGVB**(Stochastic Gradient Variational Bayes)라 한다.
$$\begin{align}
\mathcal{L}(\theta, \phi; x) &= \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x, z) - \log q_\phi(z \mid x)\bigg] \\
&= \mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x \mid z) \bigg] - KL\big (q_\phi (z \mid x) \Vert p_\theta(z) \big)
\end{align}$$

> $$\begin{align}
\mathcal{L}(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &= \mathbb{E}_{q_\boldsymbol{\phi} (z)} \bigg[ \log p_\boldsymbol{\theta} (\mathbf{x}^{(i)}, \mathbf{z}^{(i, l)}) - \log q_\boldsymbol{\phi}(\mathbf{z}^{(i, l)} \mid \mathbf{x}^{(i)})\bigg] \\
\mathcal{L}(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &\simeq \mathcal{\tilde{L}}^A(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) \\
\mathcal{\tilde{L}}^A(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &= \frac{1}{L} \sum_{l=1}^L \log p_\boldsymbol{\theta}(\mathbf{x}^{(i)}, \mathbf{z}^{(i, l)}) - \log q_\boldsymbol{\phi} (\mathbf{z}^{(i, l)} \mid \mathbf{x}^{(i)}) \\
~~~ \text{where} ~~ \mathbf{z}^{(i, l)} &= g_\phi(\boldsymbol{\epsilon}^{(i, l)}, \mathbf{x}^{(i)}) ~ \text{and} ~ \boldsymbol{\epsilon}^{(l)} \sim p(\boldsymbol{\epsilon})
\end{align}$$

> - 그런데 $q(\cdot)$와 $p(\cdot)$가 특정 분포인 경우 KL divergence를 해석적으로 구할 수 있다. 이런 경우 ELBO의 $\mathbb{E}_{q_\phi (z)} \bigg[ \log p_\theta (x \mid z) \bigg]$ 만을 샘플링으로 대체하면 되고, 이렇게 하면 앞서 경우보다 추정치량의 분산이 작아지는 효과가 있다.
> $$\begin{align}
\mathcal{L}(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &= \mathbb{E}_{q_\boldsymbol{\phi} (z)} \bigg[ \log p_\boldsymbol{\theta} (\mathbf{x}^{(i)}, \mathbf{z}^{(i, l)}) \bigg] -  KL\big (q_\phi (\mathbf{z}^{(i, l)} \mid \mathbf{x}^{(i)}) \Vert p_\theta(\mathbf{z}^{(i, l)}) \big)\\
\mathcal{L}(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &\simeq \mathcal{\tilde{L}}^A(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) \\
\mathcal{\tilde{L}}^B(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &= -  KL\big (q_\phi (\mathbf{z}^{(i, l)} \mid \mathbf{x}^{(i)}) \Vert p_\theta(\mathbf{z}^{(i, l)}) \big) + \frac{1}{L} \sum_{l=1}^L \log p_\boldsymbol{\theta}(\mathbf{x}^{(i)}, \mathbf{z}^{(i, l)}) \\
~~~ \text{where} ~~ \mathbf{z}^{(i, l)} &= g_\phi(\boldsymbol{\epsilon}^{(i, l)}, \mathbf{x}^{(i)}) ~ \text{and} ~ \boldsymbol{\epsilon}^{(l)} \sim p(\boldsymbol{\epsilon})
\end{align}$$

### AEVB(Auto-Encoding Variational Bayesian) algorithm

> - Algorithm1
 - ELBO를 최대화 하도록      
SGD나 Adagrad를 이용해 $q_\phi(\mathbf{z} \mid \mathbf{x})$의 파라미터 $\phi$와 $p_\theta(\mathbf{x}, \mathbf{z})$의 파라미터 $\theta$를 동시에 업데이트

### Variational Auto-Encoder

> - AEVB 알고리즘에서 $q_\phi(\mathbf{z} \mid \mathbf{x})$와 $p_\theta(\mathbf{x}, \mathbf{z})$에 신경망을 이용하는 모형을 VAE라 하며      
논문에서는 latent $\mathbf{z}$의 분포 $p(\mathbf{z})$가 표준정규 분포이고, $q_\phi(\mathbf{z} \mid \mathbf{x})$ 또한 어떤 정규분포 $\mathcal{N}(\boldsymbol{\mu}, \boldsymbol{\sigma}^2 \mathbf{I})$인 경우를 제시한다.

> - $p(\mathbf{z})$와 $q_\phi(\mathbf{z} \mid \mathbf{x})$가 정규분포인 경우 KL divergence를 해석적으로 구할 수 있고 그 결과는 아래와 같다.
$$\begin{align}
\mathcal{\tilde{L}}^B(\boldsymbol{\theta}, \boldsymbol{\phi}; \mathbf{x}^{(i)}) &= -  KL\big (q_\phi (\mathbf{z}^{(i, l)} \mid \mathbf{x}^{(i)}) \Vert p_\theta(\mathbf{z}^{(i, l)}) \big) + \frac{1}{L} \sum_{l=1}^L \log p_\boldsymbol{\theta}(\mathbf{x}^{(i)}, \mathbf{z}^{(i, l)}) \\
&=\frac{1}{2} \sum_{j=1}^J \bigg( 1 + \log( (\sigma_j^{(i)})^2 - (\mu_j^{(i)})^2) - (\sigma_j^{(i)})^2 \bigg) + \frac{1}{L} \sum_{l=1}^L \log p_\theta (\mathbf{x}^{(i)} \mid \mathbf{z}^{(i, l)} ) \\
~~~ \text{where} ~~ \mathbf{z}^{(i, l)} &= \boldsymbol{\mu}^{(i)} + \boldsymbol{\sigma}^{(l)} ~ \text{and} ~ \epsilon^{(l)} \sim \mathcal{N}(0, \mathbf{I}) 
\end{align}$$

### Implementation

> - https://github.com/keras-team/keras/blob/master/examples/variational_autoencoder.py